# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [39]:
#My plan
    #PREP:
    #Save all documents on my computers
    #Print instructions
    #Import packages

    #PREPARE PICTURES:
    #reformat the pictures & Create a loader (see "Loading Image Data" in "Deep Learning with PyTorch")
    #Link pics with labels

    #TRAIN A MODEL (see "Transfer Learning II" "Deep Learning with PyTorch")
    #Select a model from torchvision (VGG16)
    #Freeze
    #Create my model
    #Train the model
    #Test model (check "Transfer Learning Solution" video)


In [13]:
# All my imports
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.models as models




## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [14]:
data_dir = r'C:\Users\franc\Documents\Python\Image Classifier Project\Documents provided\flower_data'
train_dir = data_dir + r'\train'
valid_dir = data_dir + r'\valid'
test_dir = data_dir + r'\test'


In [15]:
# DONE: Define your transforms for the training, validation, and testing sets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),}

# DONE: Load the datasets with ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'valid': datasets.ImageFolder(valid_dir, transform=data_transforms['valid']),
    'test': datasets.ImageFolder(test_dir, transform=data_transforms['test']),}

# DONE: Using the image datasets and the transforms, define the dataloaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=64, shuffle=True),
    'valid': DataLoader(image_datasets['valid'], batch_size=32),
    'test': DataLoader(image_datasets['test'], batch_size=32),}


In [16]:
# DONE: Test my dataloaders

numbers_train = len(dataloaders['train'].dataset)
print(numbers_train)

numbers_valid = len(dataloaders['valid'].dataset)
print(numbers_valid)

numbers_test = len(dataloaders['test'].dataset)
print(numbers_test)


6552
818
819


In [17]:
# DONE: Check the size of my dataset
num_classes = len(image_datasets['train'].class_to_idx)
print(num_classes)

102


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [18]:
import json

# DONE: Load the label mapping from my directory
file_path = r'C:\Users\franc\Documents\Python\Image Classifier Project\Documents provided\cat_to_name.json'

with open(file_path, 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [19]:
# DONE: Build and train your network

# Load VGG16 model
model = models.vgg16(pretrained=True)

In [14]:
# Check my model loaded correctly
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [20]:
# Freezing VGG16 original parameters
for original_param in model.parameters():
    original_param.requires_grad = False

In [21]:
# Create my new classifier
model.classifier = nn.Sequential(
    nn.Linear(25088, 4096),  # Adjust input 7*7*512
    nn.ReLU(), # Activate
    nn.Dropout(0.5), # Prevent overfitting of my model
    nn.Linear(4096, 102),  # Match my number of classes
    nn.LogSoftmax(dim=1)
)


In [22]:
# Check my new classifier
print(model.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=102, bias=True)
  (4): LogSoftmax(dim=1)
)


In [23]:
# Select GPU or CPU (Udacity GPU doesn't work!)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Write the Loss function
criterion = nn.NLLLoss()

# Define the optimizer
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

# Set the number of epochs
epochs = 10
for epoch in range(epochs):
    model.train()  # Set to training mode
    running_loss = 0
    
    for inputs, labels in dataloaders['train']:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Perform a single optimization step
        optimizer.step()
        
        running_loss += loss.item()
    
    # Validation
    model.eval()  # Set to evaluation mode
    validation_loss = 0
    
    with torch.no_grad():
        for inputs, labels in dataloaders['valid']:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            validation_loss += loss.item()
    
    # Print out the average loss per epoch
    print(f"Epoch {epoch+1}/{epochs} - Training loss: {running_loss/len(dataloaders['train']):.3f} - Validation loss: {validation_loss/len(dataloaders['valid']):.3f}")


Epoch 1/10 - Training loss: 7.634 - Validation loss: 2.066
Epoch 2/10 - Training loss: 2.976 - Validation loss: 1.656
Epoch 3/10 - Training loss: 2.784 - Validation loss: 1.433
Epoch 4/10 - Training loss: 2.699 - Validation loss: 1.513
Epoch 5/10 - Training loss: 2.654 - Validation loss: 1.330
Epoch 6/10 - Training loss: 2.558 - Validation loss: 1.246
Epoch 7/10 - Training loss: 2.575 - Validation loss: 1.348
Epoch 8/10 - Training loss: 2.523 - Validation loss: 1.253
Epoch 9/10 - Training loss: 2.519 - Validation loss: 1.287
Epoch 10/10 - Training loss: 2.499 - Validation loss: 1.221


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [24]:
#DONE: Test my model

# evaluation mode
model.eval()

# Initialize variables to track accuracy
correct = 0
total = 0

# Go through the test dataset
with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        # Move data to the appropriate device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        
        # Probabilities
        ps = torch.exp(outputs)
        
        # Top class and probability per image
        top_p, top_class = ps.topk(1, dim=1)
        
        # Check predicted class matches the true labels
        equals = top_class == labels.view(*top_class.shape)
        
        # Increment correct count 
        correct += equals.sum().item()
        
        # Increment total count
        total += labels.size(0)

# Calculate accuracy
accuracy = correct / total

# Print accuracy
print(f"Test Accuracy: {accuracy:.3f}")


Test Accuracy: 0.646


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [43]:
# Create dictionary
checkpoint = {
    'model_state_dict': model.state_dict(),
    'class_to_idx': image_datasets['train'].class_to_idx,
    'num_classes': len(image_datasets['train'].classes),  # Add number of classes
    'epochs': epochs,
    'optimizer_state_dict': optimizer.state_dict()
}

# Add classifier information to the checkpoint
if 'classifier' not in checkpoint['model_state_dict']:
    checkpoint['model_state_dict']['classifier'] = model.classifier.state_dict()

# Save checkpoint
torch.save(checkpoint, checkpoint_path)



## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [47]:
def load_checkpoint(filepath):
    # Load the checkpoint
    checkpoint = torch.load(filepath)
    
    # Rebuild architecture
    model = models.vgg16(pretrained=True)

    # Freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False

    # Check the classifier architecture in the loaded checkpoint
    classifier_checkpoint = checkpoint['model_state_dict']['classifier']
    num_features_checkpoint = classifier_checkpoint[0].in_features
    
    # Check the classifier architecture in the current model
    num_features_current = model.classifier[0].in_features
    
    # Ensure consistency in the architecture
    if num_features_checkpoint != num_features_current:
        raise ValueError("Classifier architecture mismatch between checkpoint and current model.")
    
    # Define a new classifier with the correct number of output features
    classifier = nn.Sequential(
        nn.Linear(num_features_current, checkpoint['num_classes']),  # Use the number of classes from the checkpoint
        nn.LogSoftmax(dim=1)
    )

    # Replace pretrained classifier with the new one
    model.classifier = classifier

    # Load the model state dictionary
    model.load_state_dict(checkpoint['model_state_dict'])

    # Attach class_to_idx attribute to the model
    model.class_to_idx = checkpoint['class_to_idx']

    return model


In [46]:
# Check my dictionary keys

# Define the file path of the checkpoint
filepath = 'Checkpoint1.pth'

# Load the checkpoint
checkpoint = torch.load(filepath)

# Print dictionary keys
print(checkpoint.keys())


dict_keys(['model_state_dict', 'class_to_idx', 'num_classes', 'epochs', 'optimizer_state_dict'])


In [48]:
# Inspect my model
import torch

def load_checkpoint(filepath):
    # Load the checkpoint
    checkpoint = torch.load(filepath)
    
    # Extract the model state dictionary
    model_state_dict = checkpoint['model_state_dict']
    
    return model_state_dict

# Define path
checkpoint_path = 'Checkpoint1.pth'

# Load the model state dictionary from the checkpoint
model_state_dict = load_checkpoint(checkpoint_path)

# Inspect
print(model_state_dict.keys())


odict_keys(['features.0.weight', 'features.0.bias', 'features.2.weight', 'features.2.bias', 'features.5.weight', 'features.5.bias', 'features.7.weight', 'features.7.bias', 'features.10.weight', 'features.10.bias', 'features.12.weight', 'features.12.bias', 'features.14.weight', 'features.14.bias', 'features.17.weight', 'features.17.bias', 'features.19.weight', 'features.19.bias', 'features.21.weight', 'features.21.bias', 'features.24.weight', 'features.24.bias', 'features.26.weight', 'features.26.bias', 'features.28.weight', 'features.28.bias', 'classifier.0.weight', 'classifier.0.bias', 'classifier'])


In [43]:
# Check classifier
print(model_state_dict['classifier'])


OrderedDict([('0.weight', tensor([[-0.0062,  0.0028,  0.0018,  ...,  0.0009, -0.0044, -0.0033],
        [ 0.0059,  0.0009, -0.0014,  ...,  0.0018,  0.0050,  0.0014],
        [-0.0061, -0.0020,  0.0009,  ...,  0.0048,  0.0013,  0.0036],
        ...,
        [ 0.0016, -0.0037, -0.0005,  ..., -0.0029,  0.0035,  0.0055],
        [ 0.0044, -0.0061, -0.0010,  ...,  0.0048,  0.0006,  0.0049],
        [ 0.0029, -0.0004,  0.0048,  ..., -0.0048,  0.0047, -0.0040]])), ('0.bias', tensor([-4.5441e-04, -9.0557e-04, -6.1847e-03, -1.3936e-03,  4.1854e-03,
         5.9010e-05, -2.1446e-03,  1.1804e-03,  2.8827e-03,  3.1699e-03,
        -2.7115e-03,  4.9783e-03, -6.2008e-04, -3.1166e-03,  2.1098e-03,
        -6.0772e-03, -2.6067e-03,  5.5824e-03, -6.2899e-03,  4.8329e-03,
        -2.2479e-04,  3.1260e-03, -3.9603e-03, -2.9972e-03,  1.0121e-03,
        -3.3027e-04,  1.9346e-03, -5.1745e-03, -5.1939e-03, -3.6060e-03,
         1.6406e-03,  1.4117e-03,  4.9737e-03,  3.8285e-03,  6.8042e-04,
        -4.4752e

In [45]:
# Check input features
num_features_checkpoint = model_state_dict['classifier']['0.weight'].size(1)
print("Number of input features for the new classifier:", num_features_checkpoint)


Number of input features for the new classifier: 25088


In [53]:
import torch

def load_checkpoint(checkpoint_path):
    """
    Load a PyTorch model checkpoint.

    Args:
        checkpoint_path (str): Path to the checkpoint file.

    Returns:
        model: PyTorch model loaded from the checkpoint.
    """
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)
    
    # Create the model instance
    model = checkpoint
    
    return model

# Usage:
checkpoint_path = 'Checkpoint1.pth'
model = load_checkpoint(checkpoint_path)


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [60]:
from torchvision import transforms
from PIL import Image
import numpy as np

def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # DONE: Process a PIL image for use in a PyTorch model
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Open the image
    pil_image = Image.open(image)
    
    # Apply the transformations
    img_tensor = preprocess(pil_image)
    
    # Convert the tensor to a NumPy array
    np_image = np.array(img_tensor)
    
    return np_image


In [61]:
# Test my function
image_path = 'Rosepicture.jpg'
processed_image = process_image(image_path)
print(processed_image.shape)  # Check the shape of the processed image


(3, 224, 224)


In [ ]:
To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [64]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # Undo normalization
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    # Transpose the image to correct dimensions
    image = image.transpose((1, 2, 0))
    
    # Undo preprocessing
    image = std * image + mean
    
    # Clip the image
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    ax.axis('off')  # Turn off axis
    
    return ax


In [ ]:
imshow(processed_image)
plt.show()


## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes